In [1]:
from graph_of_thoughts.prompter import LanguageModelPrompter
from graph_of_thoughts.language_model import ChatGPT
lm = ChatGPT()
prompter = LanguageModelPrompter(lm, "Sort this list using merge sort algorithm")
list = "[9, 6, 7, 7, 2, 0, 2, 2, 3, 5, 0, 9, 2, 2, 4, 4, 5, 2, 5, 1, 2, 8, 3, 8, 3, 9, 6, 0, 4, 2, 2, 3]"
num_split = 4
print(f"Sort this list using merge sort algorithm {list}")




Sort this list using merge sort algorithm [9, 6, 7, 7, 2, 0, 2, 2, 3, 5, 0, 9, 2, 2, 4, 4, 5, 2, 5, 1, 2, 8, 3, 8, 3, 9, 6, 0, 4, 2, 2, 3]


In [4]:
split_prompt_raw = prompter.split_prompt({"state": f"Sort this list using merge sort algorithm\nInput", "num_split": num_split})
split_prompt = f"<Instruction>{split_prompt_raw} \nOnly output the {num_split} results in format json {{\"0\":..., \"1\": ...}} without any addtional text or thoughts.</Instruction>\nInput: {list}\nOutput:"

print(split_prompt)

<Instruction>Please split the list into four equal parts. 
Only output the 4 results in format json {"0":..., "1": ...} without any addtional text or thoughts.</Instruction>
Input: [9, 6, 7, 7, 2, 0, 2, 2, 3, 5, 0, 9, 2, 2, 4, 4, 5, 2, 5, 1, 2, 8, 3, 8, 3, 9, 6, 0, 4, 2, 2, 3]
Output:


In [5]:
split_result = lm.get_response_texts(lm.query(split_prompt))[0]
print(split_result)


{"0": [9, 6, 7, 7, 2, 0, 2, 2], "1": [3, 5, 0, 9, 2, 2, 4, 4], "2": [5, 2, 5, 1, 2, 8, 3, 8], "3": [3, 9, 6, 0, 4, 2, 2, 3]}


In [6]:
import json
split_result_dict = json.loads(split_result)

In [7]:
generate_prompt_raw = prompter.generate_prompt({"state": split_prompt_raw})


In [8]:
print(generate_prompt_raw)

Sort the given list using the merge sort algorithm.


In [19]:
generate_result_dict = {}
for k, v in split_result_dict.items():
    generate_prompt = f"""<Instruction>{generate_prompt_raw} \nOnly output the result in format json {{\"0\":}} without any addtional text or thoughts.</Instruction>\nInput: {v}\nOutput:"""
    generate_result_dict[k] = json.loads(lm.get_response_texts(lm.query(generate_prompt))[0])["0"]

    print(generate_result_dict[k])

[0, 2, 2, 2, 6, 7, 7, 9]
[0, 2, 2, 3, 4, 4, 5, 9]
[1, 2, 2, 3, 5, 5, 8, 8]
[0, 2, 2, 3, 3, 4, 6, 9]


In [20]:
print(generate_result_dict)

{'0': [0, 2, 2, 2, 6, 7, 7, 9], '1': [0, 2, 2, 3, 4, 4, 5, 9], '2': [1, 2, 2, 3, 5, 5, 8, 8], '3': [0, 2, 2, 3, 3, 4, 6, 9]}


In [24]:
aggregate_prompt_raw = prompter.aggregate_prompt({"state": generate_prompt_raw})
print(aggregate_prompt_raw)

Sort the given list using the merge sort algorithm.


In [25]:
aggregate_prompt = f"<Instruction>{aggregate_prompt_raw} \nOnly output the result in format json {{\"0\":}} without any addtional text or thoughts.</Instruction>\nInput: {generate_result_dict}\nOutput:"
print(aggregate_prompt)

<Instruction>Sort the given list using the merge sort algorithm. 
Only output the result in format json {"0":} without any addtional text or thoughts.</Instruction>
Input: {'0': [0, 2, 2, 2, 6, 7, 7, 9], '1': [0, 2, 2, 3, 4, 4, 5, 9], '2': [1, 2, 2, 3, 5, 5, 8, 8], '3': [0, 2, 2, 3, 3, 4, 6, 9]}
Output:


In [26]:
lm.get_response_texts(lm.query(aggregate_prompt))[0]

'{"0": [0, 0, 1, 2, 2, 2, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9]}'